# elasticsearch
## no sql 로 비정형화 된 시스템 임
### 데이터의 양이 많을때 빠르게 조회하여 처리하기 용이한 특징이 있음
---

* 사용하기전에 외부 라이브러리 설치

In [3]:
pip install elasticsearch==8.12.1

Note: you may need to restart the kernel to use updated packages.


## dotenv
1. env 파일에 변수를 등록함 (pc 환경마다 다른 elasticsearch 비밀번호, 경로로 인해 설정함)
2. os 와 dotenv 를 import
3. load_dotenv() 를 사용해 메모에 등록<br>
4. os.getenv("변수명") 을 활용해 .env 에 있는 변수를 불러와 적용
---
## elasticsearch 설정
1. Elasticsearch import
2. id, password 설정
3. Elasticsearch 객체 이용하여 주소, 인증키, 계정정보 삽입

In [12]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import dotenv
import os
load_dotenv()

# 첫 ID는 'elastic'이며, 비밀번호는 처음 주어진 비밀번호를 변경가능함
ELASTIC_ID = "elastic"
PASSWORD = os.getenv("PASSWORD")
ELASTIC_PASSWORD = PASSWORD
CA_CERT_PATH = os.getenv("CA_CERT_PATH")

# client instance 생성
client = Elasticsearch(
    "https://localhost:9200",  # endpoint
    ca_certs=CA_CERT_PATH, # 인증키 위치를 잡아줌
    basic_auth=(ELASTIC_ID, ELASTIC_PASSWORD),
)

# 접속이 잘 되었다면 아래 코드로 확인가능함
client.info()
# {'name': 'instance-0000000000', 'cluster_name': ...}

ObjectApiResponse({'name': 'DESKTOP-9M6U9VP', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'ohr9KyuIS_qfS8ZXm9qDFA', 'version': {'number': '8.18.0', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '04e979aa50b657bebd4a0937389308de82c2bdad', 'build_date': '2025-04-10T10:09:16.444104780Z', 'build_snapshot': False, 'lucene_version': '9.12.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## index 및 document 생성
### 인덱스는 RDBMS 의 TABLE 같은 존재
1. 선언한 client 를 사용해 indices.create(index="인덱스 이름") 로 생성
2. client.index() 로 document, field 설정
* document 는 RDBMS의 행 개념
* field 는 RDBMS 의 컬럼 개념<br>
※ RDBMS 와 다르게 document 를 설정했어도 다른 값이 있다면 추가되는 차이가있음

In [ ]:
# result = client.indices.create(index="my_index2")
# print(result)

result = client.index(
    index="my_index2",
    id="my_document_id",
    document={"field1": "value1", "field2": "value2"},
)
print(result)

result = client.get(index="my_index2", id="my_document_id")
print(result)

{'_index': 'my_index2', '_id': 'my_document_id', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'my_index2', '_id': 'my_document_id', '_version': 1, '_seq_no': 3, '_primary_term': 1, 'found': True, '_source': {'filed1': 'value1', 'filed2': 'value2'}}


## elasticsearch 의 조회 및 문서 관리
### 조회
```python
client.search(index="인덱스이름", query={"match":{"필드이름":"값이름"}})
```
### 문서 수정
```python
client.update(index="인덱스이름", id = "문서이름", doc={"수정할 필드이름":"수정할 값 이름"})
```
* 문서를 수정할때 없는 필드는 새로 추가됨
### 문서 삭제
```python
client.delete(index="인덱스이름", id="문서이름")
```

In [ ]:
result = client.search(index="my_index2", query={"match": {"field1": "value1"}})
print(result)

result = client.update(
    index="my_index2",
    id="my_document_id",
    doc={"field1": "new values", "new_field": "new value2"},
)
print(result)

# result = client.delete(
#     index="my_index2",
#     id="my_document_id",
# )

# print(result)

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'_index': 'my_index2', '_id': 'my_document_id', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}


In [19]:
result = client.get(index="my_index2", id="my_document_id")
result["_source"]

{'filed1': 'new values', 'filed2': 'value2', 'new_field': 'new value2'}